# Caribbean shallow hardbottom and coral

This is code for Southeast Conservation Blueprint 2023. Written by Rua Mordecai

## Import libraries and define variables

In [87]:
import os
import arcpy

In [88]:
# define spatial reference, workspaces, and source data location
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\CaribbeanBlueprint2023_FinalIndicators\CaribbeanBlueprint2023_FinalIndicators\SpatialData"
benthicPR = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\pr_mar_benhab4m_2021_EEZs_gridcode.tif"
benthicVI = r"D:\CaribbeanData\TNC\BenthicCondition\BENTHIC_CONDITION_PRVI\usvi_mar_benhab4m_2021_EEZs_gridcode.tif"
refugia = r"D:\CaribbeanData\TNC\CoralRefugia_122021\CoralRefugia_122021.shp"

In [89]:
# define final output name
IndicatorFileName = "CaribbeanShallowHardbottomAndCoral.tif"

In [90]:
# define raster used for cell size, extent, and snapping
CaribbeanRaster= r"D:\Blueprint\2023\extent\VIPR_Extent_v6.tif"

In [91]:
# Sometimes arcpro is fussy about not overwriting things
arcpy.env.overwriteOutput = True

## Start analysis

In [92]:
# Set the source workspace
arcpy.env.workspace = OutputWorkspace

In [93]:
# convert coral refugia rank to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
    arcpy.conversion.FeatureToRaster(in_features=refugia, field="rank_PRU", out_raster="refugia.tif", cell_size=CaribbeanRaster)

In [94]:
# reclass to get look at below average and above average resilience.Zero in the rank meant it wasn't assessed so it's scored
# the same as NoData
out_raster = arcpy.sa.Reclassify("refugia.tif", "Value", "0 0;1 775 10;776 1552 100;NoData 0", "DATA"); out_raster.save("refugiaBin.tif")

In [95]:
arcpy.management.MosaicToNewRaster(benthicPR+";"+benthicVI, OutputWorkspace, "vipr_mar_benhab4m_2021_EEZs_gridcode.tif", None, "8_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Coral\\working\\vipr_mar_benhab4m_2021_EEZs_gridcode.tif'>

In [96]:
# reproject and convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=CaribbeanRaster, snapRaster=CaribbeanRaster, cellSize=CaribbeanRaster):
        arcpy.management.Resample("vipr_mar_benhab4m_2021_EEZs_gridcode.tif", "benthic.tif", "30 30", "MAJORITY")

In [97]:
# put benthic data in to indicator categories. Boulders and rocks (11) do not occur in this extent but would be a category before
# hardbottom sparse algae
out_raster = arcpy.sa.Reclassify("benthic.tif", "Value", "1 5 3;6 2;7 1;8 10 0;11 1;12 13 0", "DATA"); out_raster.save("benthicBin.tif")

In [98]:
# Combine rasters for indicator categories
out_raster = arcpy.sa.Plus("benthicBin.tif", "refugiaBin.tif"); out_raster.save("PlusRefugia.tif")

In [99]:
out_raster = arcpy.sa.Reclassify("PlusRefugia.tif", "Value", "0 0;1 1;2 2;3 4;10 0;11 1;12 2;13 3;100 0;101 1;102 2;103 5", "DATA"); out_raster.save("PlusRefugiaBin.tif")

In [100]:
# clip raster to Caribbean extent
out_raster = arcpy.sa.ExtractByMask("PlusRefugiaBin.tif", CaribbeanRaster); out_raster.save(IndicatorFileName)

## Finalize indicator

In [107]:
# set code block for legend
codeblock = """
def Reclass(value):
    if value == 0:
        return '0 = Not coral or hardbottom'
    elif value == 1:
        return '1 = Hardbottom with dense algae' 
    elif value == 2:
        return '2 = Hardbottom with sparse algae'    
    elif value == 3:
        return '3 = Coral with below average climate resilience'
    elif value == 4:
        return '4 = Coral with unknown climate resilience'
    elif value == 5:
        return '5 = Coral with above average climate resilience'
"""

In [108]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(IndicatorFileName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Caribbean\\Indicators\\Coral\\working\\CaribbeanShallowHardbottomAndCoral.tif'>

In [93]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 39
	if Value == 3:
		return 168
	if Value == 2:
		return 255
	if Value == 1:
		return 255
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 96
	if Value == 4:
		return 130
	if Value == 3:
		return 137
	if Value == 2:
		return 161
	if Value == 1:
		return 202
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 166
	if Value == 4:
		return 196
	if Value == 3:
		return 184
	if Value == 2:
		return 142
	if Value == 1:
		return 191
	if Value == 0:
		return 255
	else:
		return 255
		
"""

In [94]:
# calculate Red field
arcpy.management.CalculateField(IndicatorFileName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(IndicatorFileName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(IndicatorFileName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\CaribbeanBlueprint2023_FinalIndicators\\CaribbeanBlueprint2023_FinalIndicators\\SpatialData\\CaribbeanShallowHardbottomAndCoral.tif'>